# LARS Tutorial

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import os
import argparse

# from utils import progress_bar // for linux

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [ ]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Model
print('==> Building model..')
net = models.resnet50()
# net = models.alexnet()
# net = models.vgg16()
# net = models.squeezenet1_0()
# net = models.densenet161()
# net = models.inception_v3()
# net = models.googlenet()
# net = models.shufflenet_v2_x1_0()
# net = models.mobilenet_v2()
# net = models.resnext50_32x4d()
# net = net.to(device)
if device == 'cuda':
    net.cuda()
#     range_of_cuda_device = range(torch.cuda.device_count())
    net = torch.nn.DataParallel(net, device_ids=[1])
    cudnn.benchmark = True

In [ ]:
criterion = nn.CrossEntropyLoss()
start_lr = 0.1
optimizer = optim.SGD(net.parameters(), lr=start_lr, momentum=0.9, weight_decay=5e-4)
lambda1 = lambda epoch: (1 - epoch / 200) ** 2
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])

In [ ]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(device=1), targets.cuda(device=1)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print('\r[%d/%d] Loss: %.3f | Acc: %.3f%% (%d/%d)' % (batch_idx, len(trainloader), train_loss/(batch_idx+1), 100.*correct/total, correct, total), end='')

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(device=1), targets.cuda(device=1)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print('\r[%d/%d] Loss: %.3f | Acc: %.3f%% (%d/%d)' % (batch_idx, len(testloader), test_loss/(batch_idx+1), 100.*correct/total, correct, total), end='')

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

In [ ]:
for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    print()
    test(epoch)
    scheduler.step()